In [116]:
SAMPLE=False

def load_input(file):
  with open(file) as f:
    rules,messages=[l for l in f.read().split('\n\n')]

    rules=dict((int(y[0]), [(int(z) if z.isdigit() else z) for z in y[1].split(' ')]) for y in [x.split(': ') for x in rules.splitlines()])
    messages=messages.splitlines()
    return rules,messages

rules,messages=load_input('input_sample.txt' if SAMPLE else 'input.txt')

if SAMPLE: print(rules); print(messages)

### Part 1

In [117]:
def processKey(key: int = 0) -> str:
  """Recursive replace dict with true ab values"""
  val = rules[key]

  if not isinstance(val, str):
    for idx,item in enumerate(val):
      if isinstance(item, int):
        val[idx] = processKey(item)
    # at this point the list should all be strings
    rules[key] = "(" + "".join(val).replace('"','') + ")"

  return rules[key]

processKey()

# Do Regex matching
import re
tester: re.Pattern = re.compile(rules[0])
part1 = sum(1 if re.fullmatch(tester, m) else 0 for m in messages)

assert(part1 == 2 if SAMPLE else 230)

part1

230

### Part 2


The solution requires using more advanced regex.
The required new rules:

> 8: 42 | 42 8

> 11: 42 31 | 42 11 31

can be written as such in a regular expression format:

```py
rules[8] = [42,'+']
rules[11] = [42,'{n}',31,'{n}']
```
However the new rule for `11` is not part of a [regular grammar](https://en.wikipedia.org/wiki/Regular_grammar#Mixing_left-regular_and_right-regular_rules) (namely that it includes both a right-regular and left-regular rule; see link). In fact it includes the paradigmatic non-regular linear form:



Because it is not part of a regular grammar, it can't be defined by just one __regular__ expresssion (regex), either. (Regexes seem to operate on right-regular grammars.)

But it is still a **linear grammar** rule, so there is a way to solve for rule #11 by testing reasonably for all values of `n`, `n >= 1`, in the regex:

```regex
/42{n}31{n}/

```

In [115]:
rules,messages=load_input('input_sample_part2.txt' if SAMPLE else 'input.txt')
if SAMPLE: print(rules); print(messages)

rules[8] = [42,'+']
rules[11] = [42,'{n}',31,'{n}']

processKey()

part2 = 0

# "5" was chosen empirically as no nonzero summands appeared after i=5
for i in range(1,5):
  tester: re.Pattern = re.compile(rules[0].replace("{n}","{" + str(i) + "}"))
  part2 += sum(1 if re.fullmatch(tester, m) else 0 for m in messages)

assert(part2 == 12 if SAMPLE else 342)

part2

341